<a href="https://colab.research.google.com/github/Abdulmathin-shaik/Kaggle/blob/main/FMNIST_100th_time_because_you_suck!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# transform_func = transforms.Compose([ToTensor()])


In [4]:
train_fmnist = datasets.FashionMNIST(root='data',train=True,download=True,transform=ToTensor())
test_fmnist = datasets.FashionMNIST(root='data',train=False,transform=ToTensor())

100%|██████████| 26.4M/26.4M [00:02<00:00, 13.1MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.90MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.66MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
train_dataloader = DataLoader(train_fmnist,batch_size=64,shuffle=True)
test_dataloader = DataLoader(test_fmnist,batch_size=64)

In [64]:
class simple_model(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3),
            nn.BatchNorm2d(64),
            # nn.Dropout2d(0.5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64,64,3),
            # nn.Dropout2d(0.5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self,x):
        # x = self.flatten(x)
        x = self.stack(x)
        return x

In [65]:
model = simple_model().to(device)

In [66]:
model(torch.randn(64,1,28,28).to(device)).shape

torch.Size([64, 10])

In [67]:
epochs = 20
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-04)

In [68]:
train_losses,test_losses = [],[]
train_accs,test_accs =[],[]
for epoch in range(epochs):
    train_loss=0
    for batch,(x,y) in enumerate(train_dataloader):
        model.train()
        x,y = x.to(device),y.to(device)
        pred = model(x)
        loss = loss_fn(pred.squeeze(0),y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    epoch_train_loss = train_loss/len(train_dataloader)
    train_losses.append(epoch_train_loss)
    print(f'Training loss for epoch {epoch}: {epoch_train_loss}')

    with torch.no_grad():
        test_loss,test_acc=0,0
        for batch,(x,y) in enumerate(test_dataloader):
            model.eval()
            x,y = x.to(device),y.to(device)
            test_pred = model(x)
            loss = loss_fn(test_pred,y)
            test_loss += loss.item()
            test_acc += ((y==test_pred.argmax(dim=1)).sum().item())/len(y)
        epoch_test_loss = train_loss/len(test_dataloader)
        epoch_acc = test_acc/len(test_dataloader)
        test_losses.append(epoch_test_loss)
        test_accs.append(epoch_acc)

        print(f'Training loss for epoch {epoch}: {epoch_test_loss} and test accuracy is {epoch_acc}')

Training loss for epoch 0: 0.8195766402460111
Training loss for epoch 0: 4.896578907966614 and test accuracy is 0.8003582802547771
Training loss for epoch 1: 0.47654389609088266
Training loss for epoch 1: 2.847122130785019 and test accuracy is 0.8269307324840764
Training loss for epoch 2: 0.41116491698824775
Training loss for epoch 2: 2.456513962643162 and test accuracy is 0.8429538216560509
Training loss for epoch 3: 0.3753816590253224
Training loss for epoch 3: 2.2427260902277224 and test accuracy is 0.8568869426751592
Training loss for epoch 4: 0.3477785639099475
Training loss for epoch 4: 2.077810783105291 and test accuracy is 0.8597730891719745
Training loss for epoch 5: 0.329400286396175
Training loss for epoch 5: 1.9680093543924344 and test accuracy is 0.8664410828025477
Training loss for epoch 6: 0.31236071805201615
Training loss for epoch 6: 1.8662060734572683 and test accuracy is 0.869327229299363
Training loss for epoch 7: 0.2989325499706177
Training loss for epoch 7: 1.7859

In [ ]:
plt.plot(train_losses,label='train_loss')
plt.plot(test_losses,label='test_loss')
plt.legend()

In [ ]:
# plt.plot(train_accs,label='train_acc')
plt.plot(test_accs,label='test_acc')
plt.legend()

In [49]:
x,y=next(iter(test_dataloader))

In [50]:
final_preds = model(x.to(device))

In [51]:
final_preds.argmax(dim=1)

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 5, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 5,
        1, 6, 6, 0, 9, 4, 8, 8, 1, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 9, 4, 7, 2, 1,
        4, 6, 4, 6, 5, 6, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5], device='cuda:0')

In [52]:
y

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5])

In [55]:
# model1_accurcy =
(final_preds.argmax(dim=1)==y.to(device)).sum().item()/len(y)

0.828125

In [ ]:
!pip install timm

In [70]:
import timm

In [99]:
tl_model = timm.create_model('resnet50',pretrained=True)

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [105]:
tl_model.fc = nn.Linear(512,10)

In [106]:
tl_model = tl_model.to(device)

In [107]:
tl_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [102]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),  # Resize for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalize for 3 channels
])


In [103]:
epochs = 20
loss_fn = nn.CrossEntropyLoss()
tl_optimizer = torch.optim.Adam(tl_model.parameters(),lr=1e-04)

In [108]:
train_losses,test_losses = [],[]
train_accs,test_accs =[],[]
for epoch in range(epochs):
    train_loss=0
    for batch,(x,y) in enumerate(train_dataloader):
        model.train()
        x,y = x.to(device),y.to(device)
        pred = tl_model(x)
        loss = loss_fn(pred.squeeze(0),y)
        loss.backward()
        tl_optimizer.step()
        tl_optimizer.zero_grad()
        train_loss += loss.item()
    epoch_train_loss = train_loss/len(train_dataloader)
    train_losses.append(epoch_train_loss)
    print(f'Training loss for epoch {epoch}: {epoch_train_loss}')

    with torch.no_grad():
        test_loss,test_acc=0,0
        for batch,(x,y) in enumerate(test_dataloader):
            model.eval()
            x,y = x.to(device),y.to(device)
            test_pred = tl_model(x.repeat(1,3,1,1))
            loss = loss_fn(test_pred,y)
            test_loss += loss.item()
            test_acc += ((y==test_pred.argmax(dim=1)).sum().item())/len(y)
        epoch_test_loss = train_loss/len(test_dataloader)
        epoch_acc = test_acc/len(test_dataloader)
        test_losses.append(epoch_test_loss)
        test_accs.append(epoch_acc)

        print(f'Training loss for epoch {epoch}: {epoch_test_loss} and test accuracy is {epoch_acc}')

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[64, 1, 28, 28] to have 3 channels, but got 1 channels instead

In [ ]:
alex_net = torchvision.models.vgg16(pretrained=True).to(device)